In [1]:
import numpy as np
import pandas as pd
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, RepeatVector, Embedding, TimeDistributed, Dense, Dropout, Conv1D, GRU, BatchNormalization, Activation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
from sklearn.metrics import classification_report, confusion_matrix
import ast
from itertools import cycle

In [2]:
df = pd.read_csv('Bruteforce_CWE-307.csv')

In [3]:
attack = []
normal = []
for i in range(len(df)):
    calls = ast.literal_eval(df.iloc[i]['syscalls'])
    check = df.iloc[i]['is_exploit']
    
    temp_list = []
    for j in range(len(calls)):
        temp_list.append(calls[j]['name'])
    if check:
        attack.append(temp_list)
    else:
        normal.append(temp_list)

both_lists = attack + normal
tokenizer = Tokenizer()
tokenizer.fit_on_texts(both_lists)
word_index = tokenizer.word_index

X_train = normal[:500]
X_val = normal[500:750]
X_test = normal[750:] + attack
tokened_Xtrain = tokenizer.texts_to_sequences(X_train)
tokened_Xtest = tokenizer.texts_to_sequences(X_test)
tokened_Xval = tokenizer.texts_to_sequences(X_val)
max_length = 5000
X_train_padded = pad_sequences(tokened_Xtrain, maxlen=max_length, padding='post')
X_test_padded = pad_sequences(tokened_Xtest, maxlen=max_length, padding='post')
X_val_padded = pad_sequences(tokened_Xval, maxlen=max_length, padding='post')

K = len(word_index) + 1

In [8]:
model = Sequential()
model.add(Embedding(input_dim=K, output_dim=400, input_length=None))
model.add(Conv1D(filters=16, kernel_size=3, padding="causal"))
for i, dilation_rate in zip(range(10), cycle(dilation_rates)):
    model.add(WaveNetBlock(dilation_rate=dilation_rate))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dense(K, activation='softmax'))  
model.add(Conv1D(
        bias_initializer="zeros",
        filters=16,
        kernel_initializer="glorot_uniform",
        kernel_size=3,
        padding="causal",
    ))
model.add(BatchNormalization())
model.add(Conv1D(
        activation="softmax",
        bias_initializer="zeros",
        filters=K,
        kernel_initializer="glorot_uniform",
        kernel_size=1,
    ))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_crossentropy'])

In [5]:
dilation_rates = tuple(2 ** x for x in range(10))
print(dilation_rates)

(1, 2, 4, 8, 16, 32, 64, 128, 256, 512)


In [7]:
class WaveNetBlock(tensorflow.keras.layers.Layer):
    """
    Implements the basic building block of the WaveNet architecture:
        https://arxiv.org/abs/1609.03499
    """

    def __init__(
        self,
        activation="tanh",
        bias_initializer="zeros",
        dilation_rate=1,
        filters=16,
        gate_activation="sigmoid",
        kernel_initializer="glorot_uniform",
        kernel_size=3,
        padding="causal",
        **kwargs,
    ):
        """
        Args:
            activation: (str or Callable)
                Name of a keras activation function or an instance of a keras/Tensorflow activation function.
                Applied to the non-gate branch of a gated activation unit.
            bias_initializer: (str or Callable)
                Name or instance of a keras.initializers.Initializer.
            dilation_rate: (int)
                Dilation rate used in convolutions.
            filters: (int)
                Number of filters used in convolutions.
            kernel_initializer: (str or Callable)
                Name or instance of a keras.initializers.Initializer.
            gate_activation: (str or Callable)
                Name of a keras activation function or an instance of a keras/Tensorflow activation function.
                Applied to the gate branch of a gated activation unit
            kernel_size: (tuple[int] or int)
                Dimensions of the convolution filters.
            residual_merge: (keras.layers.Layer)
                Keras layer that merges the input and output branches of a residual block.
        """
        self.activation = activation
        self.bias_initializer = bias_initializer
        self.dilation_rate = dilation_rate
        self.filters = filters
        self.gate_activation = gate_activation
        self.kernel_initializer = kernel_initializer
        self.kernel_size = kernel_size
        self.padding = padding

        self.value_branch = None
        self.gate_branch = None
        self.skip_out = None

        super().__init__(**kwargs)

    def build(self, input_shape):
        self.value_branch = Conv1D(
            activation=self.activation,
            bias_initializer=self.bias_initializer,
            dilation_rate=self.dilation_rate,
            filters=self.filters,
            kernel_initializer=self.kernel_initializer,
            kernel_size=self.kernel_size,
            padding=self.padding,
        )
        self.value_branch.build(input_shape)
        self._trainable_weights.extend(self.value_branch.trainable_weights)

        self.gate_branch = Conv1D(
            activation=self.gate_activation,
            bias_initializer=self.bias_initializer,
            dilation_rate=self.dilation_rate,
            filters=self.filters,
            kernel_initializer=self.kernel_initializer,
            kernel_size=self.kernel_size,
            padding=self.padding,
        )
        self.gate_branch.build(input_shape)
        self._trainable_weights.extend(self.gate_branch.trainable_weights)

        self.skip_out = Conv1D(
            bias_initializer=self.bias_initializer,
            dilation_rate=self.dilation_rate,
            filters=self.filters,
            kernel_initializer=self.kernel_initializer,
            kernel_size=1,
        )
        self.skip_out.build(self.value_branch.compute_output_shape(input_shape))
        self._trainable_weights.extend(self.skip_out.trainable_weights)

        super().build(input_shape)
        
    def call(self, inputs, **kwargs):
        value = self.value_branch(inputs)
        gate = self.gate_branch(inputs)
        gated_value = tensorflow.keras.layers.multiply([value, gate])
        skip_out = self.skip_out(gated_value)
        return tensorflow.keras.layers.concatenate([inputs, skip_out])

    def compute_output_shape(self, input_shape):
        output_shape = list(input_shape)
        output_shape[-1] += self.filters
        return tuple(output_shape)




In [9]:
K = len(word_index) + 1
print(train_inputs.shape)

NameError: name 'train_inputs' is not defined

In [23]:
def add_train_labels(x):
        return x[:-1], x[1:]
K = len(word_index) + 1
train = [add_train_labels(seq) for seq in X_train_padded]
val = [add_train_labels(seq) for seq in X_val_padded]

train_inputs, train_targets = zip(*train)
val_inputs, val_targets = zip(*val)

# Convert to numpy arrays
train_inputs = np.array(train_inputs)
train_targets = np.array(train_targets)
val_inputs = np.array(val_inputs)
val_targets = np.array(val_targets)

early_stopping = EarlyStopping(
    monitor='val_loss',  # Metric to monitor
    patience=3,          # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore the model weights from the epoch with the best value of the monitored metric
)


In [24]:
model.fit(
            train_inputs,
            train_targets,
            validation_data=(val_inputs, val_targets),
            epochs=150,
            verbose=2,
            shuffle=True,
            callbacks=[early_stopping],
        )

Epoch 1/150
27/27 - 71s - loss: 0.9021 - sparse_categorical_crossentropy: 0.9021 - val_loss: 0.8187 - val_sparse_categorical_crossentropy: 0.8187 - 71s/epoch - 3s/step
Epoch 2/150
27/27 - 64s - loss: 0.3711 - sparse_categorical_crossentropy: 0.3711 - val_loss: 0.5402 - val_sparse_categorical_crossentropy: 0.5402 - 64s/epoch - 2s/step
Epoch 3/150
27/27 - 64s - loss: 0.3191 - sparse_categorical_crossentropy: 0.3191 - val_loss: 0.4125 - val_sparse_categorical_crossentropy: 0.4125 - 64s/epoch - 2s/step
Epoch 4/150
27/27 - 63s - loss: 0.2911 - sparse_categorical_crossentropy: 0.2911 - val_loss: 0.3408 - val_sparse_categorical_crossentropy: 0.3408 - 63s/epoch - 2s/step
Epoch 5/150
27/27 - 63s - loss: 0.2721 - sparse_categorical_crossentropy: 0.2721 - val_loss: 0.3028 - val_sparse_categorical_crossentropy: 0.3028 - 63s/epoch - 2s/step
Epoch 6/150
27/27 - 63s - loss: 0.2587 - sparse_categorical_crossentropy: 0.2587 - val_loss: 0.2802 - val_sparse_categorical_crossentropy: 0.2802 - 63s/epoch - 

In [12]:
val = X_val_padded
y_val = model.predict(val)
sums_val = np.array([-np.log(pred.max(axis=-1)).sum(axis=-1) for pred in y_val])
threshold = np.mean(sums_val) + 2*np.std(sums_val)
print(threshold)

8/8 [==============================] - 4s 426ms/step
622.1158905029297


In [13]:
test = X_test_padded
print(len(test))
y_pred = model.predict(test)
sums = np.array([-np.log(pred.max(axis=-1)).sum(axis=-1) for pred in y_pred])

342
11/11 [==============================] - 5s 434ms/step


In [15]:
threshold = np.mean(sums_val) + 2*np.std(sums_val)
successes = 0
total = len(test)
pred = []
norms_len = total - len(attack)
testY = []
for i in range(total):
    if sums[i] <= threshold:
        pred.append(0)
    else:
        pred.append(1)
    if i < norms_len:
        testY.append(0)
        if sums[i] <= threshold:
            successes += 1
    else:
        testY.append(1)
        if sums[i] > threshold:
            successes += 1
print(successes)
print(total)
print(successes / total)

323
342
0.9444444444444444


In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [17]:
precision = precision_score(testY, pred, pos_label=0)
recall = recall_score(testY, pred, pos_label=0)
f1 = f1_score(testY, pred, pos_label=0)
print("Precision: ",precision)
print("Recall: ",recall)
print("f1: ",f1)

Precision:  0.9554655870445344
Recall:  0.9672131147540983
f1:  0.9613034623217923


In [18]:
df2 = pd.read_csv('CVE-2012-2122.csv')
attack = []
normal = []
for i in range(len(df2)):
    calls = ast.literal_eval(df2.iloc[i]['syscalls'])
    check = df2.iloc[i]['is_exploit']
    
    temp_list = []
    for j in range(len(calls)):
        temp_list.append(calls[j]['name'])
    if check:
        attack.append(temp_list)
    else:
        normal.append(temp_list)

both_lists = attack + normal
tokenizer = Tokenizer()
tokenizer.fit_on_texts(both_lists)
word_index = tokenizer.word_index

X_train = normal[:850]
X_val = normal[850:1050]
X_test = normal[1050:] + attack
tokened_Xtrain = tokenizer.texts_to_sequences(X_train)
tokened_Xtest = tokenizer.texts_to_sequences(X_test)
tokened_Xval = tokenizer.texts_to_sequences(X_val)
max_length = 7500
X_train_padded = pad_sequences(tokened_Xtrain, maxlen=max_length, padding='post')
X_test_padded = pad_sequences(tokened_Xtest, maxlen=max_length, padding='post')
X_val_padded = pad_sequences(tokened_Xval, maxlen=max_length, padding='post')

K = len(word_index) + 1

In [19]:
model.fit(
            train_inputs,
            train_targets,
            validation_data=(val_inputs, val_targets),
            epochs=150,
            verbose=2,
            shuffle=True,
            callbacks=[early_stopping],
        )

Epoch 1/150
16/16 - 25s - loss: 0.2098 - sparse_categorical_crossentropy: 0.2098 - val_loss: 0.2238 - val_sparse_categorical_crossentropy: 0.2238 - 25s/epoch - 2s/step
Epoch 2/150
16/16 - 24s - loss: 0.2092 - sparse_categorical_crossentropy: 0.2092 - val_loss: 0.2240 - val_sparse_categorical_crossentropy: 0.2240 - 24s/epoch - 2s/step
Epoch 3/150
16/16 - 24s - loss: 0.2086 - sparse_categorical_crossentropy: 0.2086 - val_loss: 0.2231 - val_sparse_categorical_crossentropy: 0.2231 - 24s/epoch - 2s/step
Epoch 4/150
16/16 - 25s - loss: 0.2081 - sparse_categorical_crossentropy: 0.2081 - val_loss: 0.2239 - val_sparse_categorical_crossentropy: 0.2239 - 25s/epoch - 2s/step
Epoch 5/150
16/16 - 24s - loss: 0.2074 - sparse_categorical_crossentropy: 0.2074 - val_loss: 0.2226 - val_sparse_categorical_crossentropy: 0.2226 - 24s/epoch - 2s/step
Epoch 6/150
16/16 - 25s - loss: 0.2064 - sparse_categorical_crossentropy: 0.2064 - val_loss: 0.2221 - val_sparse_categorical_crossentropy: 0.2221 - 25s/epoch - 

In [25]:
val = X_val_padded
y_val = model.predict(val)
sums_val = np.array([-np.log(pred.max(axis=-1)).sum(axis=-1) for pred in y_val])
threshold = np.mean(sums_val) + 2*np.std(sums_val)
print(threshold)

7/7 [==============================] - 5s 647ms/step
1111.7434692382812


In [26]:
test = X_test_padded
print(len(test))
y_pred = model.predict(test)
sums = np.array([-np.log(pred.max(axis=-1)).sum(axis=-1) for pred in y_pred])

345
11/11 [==============================] - 8s 690ms/step


In [30]:
threshold = np.mean(sums_val) + np.std(sums_val)
successes = 0
total = len(test)
pred = []
norms_len = total - len(attack)
testY = []
for i in range(total):
    if sums[i] <= threshold:
        pred.append(0)
    else:
        pred.append(1)
    if i < norms_len:
        testY.append(0)
        if sums[i] <= threshold:
            successes += 1
    else:
        testY.append(1)
        if sums[i] > threshold:
            successes += 1
print(successes)
print(total)
print(successes / total)

262
345
0.7594202898550725
